## RunnableLambda 객체 사용방법

In [4]:
from langchain_core.runnables import RunnableLambda

def func(x):
    return  x*2

runnable_1 = RunnableLambda(func)

print(runnable_1.invoke(10))

20


In [6]:
# RunnableLambda 객체 사용
runnable_1 = RunnableLambda(lambda x: x*2)
print(runnable_1.invoke(10))

20


In [9]:
# runnable batch
runnable_3 = RunnableLambda(func)

print(runnable_3.batch([10, 20, 30]))

[20, 40, 60]


In [ ]:
def gen(x):
    for y in x :
        yield y * 2
    

# runnable 
runnable_4 = RunnableLambda(gen)
for chunk in runnable_4.stream(range(10)):
    print(chunk)


In [11]:
def gen(x):
    for y in x :
        yield y * 2
    

# runnable 
runnable_4 = RunnableLambda(gen)
for chunk in runnable_4.stream(range(10)):
    print(chunk)


0
2
4
6
8
10
12
14
16
18


## 순차적 실행하게 하는 방법

In [12]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

chain = r1 | r2

chain.invoke(10)



32

# 

In [13]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

32

In [16]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 2)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 12}

In [17]:
%pip install -qU grandalf

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 예제

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "질의 내용"
        } | prompt
print(chain.invoke({}))

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [28]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = ({
            # vector 컨텐스트 들어갈 부분
            "context": RunnableLambda(lambda x: "Vector Search에 의한 컨텐스트 내용"), 
            "query": RunnablePassthrough()
        } | prompt)
# 이미지를 넣는 부분
print(chain.invoke("질문입니다.").messages[0].content)


    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    Vector Search에 의한 컨텐스트 내용
    질문:
    질문입니다.



- Runnable 객체를 만들면 pipeline에 태울 수 있다는 것을 아는게 중요함.